In [4]:
#######################
# Elastic Net Regression #
#######################
import numpy as np
import pandas as pd
import datetime
import time
import os,sys
import gc
from sklearn import *
import lightgbm

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l2/kfold' % DataBaseDir
OutputDir = '%s/l3/kfold' % DataBaseDir
kfold = 5
InputCols = ['lgb_l1', 'lgb_l2', 'lgb_huber', 'lgb_fair', 'en', 'lassolars']
#InputCols = ['lgb_l2', 'etr', 'knn']
strategy = 'en'
start_time = datetime.datetime.now()
#### load data
valid_dfs = []
holdout_dfs = []
test_dfs = []
join_keys = ['air_store_id', 'visit_date']
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    for i in range(len(InputCols)):
        valid = pd.read_csv('%s/valid_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        holdout = pd.read_csv('%s/holdout_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        test = pd.read_csv('%s/test_%s.csv' % (FoldInputDir, InputCols[i]), parse_dates= ['visit_date']).reset_index(drop= True)
        if(i == 0):
            FoldValid = valid
            FoldHoldout = holdout
            FoldTest = test
        else:
            FoldValid = FoldValid.merge(valid[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)
            FoldHoldout = FoldHoldout.merge(holdout[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)
            FoldTest = FoldTest.merge(test[[c for c in test.columns if(c not in ['id', 'visitors'])]], how= 'left', on= join_keys)  
    FoldValid['fold'] = fold
    valid_dfs.append(FoldValid)
    holdout_dfs.append(FoldHoldout)
    test_dfs.append(FoldTest)
TrainData = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('----------------- CHECK -------------')
print('\n============================ TrainData')
print(TrainData.head(5))
print('\n============================ HoldoutData')
print(holdout_dfs[0].head(5))
print('\n============================ HoldoutData')
print(test_dfs[0].head(5))
print('-------------------------------------')
##### model selection with CV
# score
cv_score = .0
holdout_score = .0
# predict
y_test_pred = 0
for fold in range(kfold):
    FoldData = {
        'train': TrainData[TrainData['fold'] != fold],
        'valid': TrainData[TrainData['fold'] == fold],
        'holdout': holdout_dfs[fold],
        'test': test_dfs[fold]
    }
    col = [c for c in FoldData['train'].columns if c not in ['id', 'air_store_id', 'visit_date', 'visitors', 'hpg_store_id', 'fold']]
    # train
    model = linear_model.ElasticNet(alpha= 0.00001, l1_ratio= 0.2, max_iter= 400, tol= 1e-4, selection= 'random', random_state= 2017)
    model.fit(FoldData['train'][col].astype(np.float32, copy=False), FoldData['train']['visitors'].values.astype(np.float32, copy=False))
    # for valid
    FoldData['valid'][strategy] = model.predict(FoldData['valid'][col])
    rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
    cv_score += rmsle_valid
    # for holdout
    FoldData['holdout'][strategy] = model.predict(FoldData['holdout'][col])
    rmsle_holdout = RMSLE(FoldData['holdout']['visitors'].values, FoldData['holdout'][strategy])
    holdout_score += rmsle_holdout
    # for test
    FoldData['test'][strategy] = model.predict(FoldData['test'][col])
    y_test_pred += FoldData['test'][strategy]

    print('fold %s: valid score %.6f, holdout score %.6f, valid length %s' % (fold, rmsle_valid, rmsle_holdout, len(FoldData['valid'])))  
    #### output
    FoldOutputDir = '%s/%s' % (OutputDir, fold)
    if(os.path.exists(FoldOutputDir) == False):
        os.makedirs(FoldOutputDir)
    for mod in FoldData.keys():
        if(mod == 'train'):
            continue
        OutCols = []
        if(mod == 'test'):
            OutCols.append('id')
        OutCols.extend(['air_store_id', 'visit_date', 'visitors', strategy])
        OutputFile = '%s/%s_%s.csv' % (FoldOutputDir, mod, strategy)
        OutFoldData = FoldData[mod][OutCols]
        OutFoldData.to_csv(OutputFile, index= False)
    print('saving for %sth fold data done.' % (fold))
    
y_test_pred /= kfold  # Average test set predictions
cv_score /= kfold # Average valid set predictions
holdout_score /= kfold # Average holdout set predictions

# Create submission file
sub = pd.DataFrame()
sub['id'] = test_dfs[0]['id']
sub['visitors'] = np.expm1(y_test_pred)
OutputFileName = '%s_submit_%s' % (strategy, datetime.datetime.now().strftime("%Y-%m-%d"))
SubmitDir = '%s/l3/submit' % DataBaseDir
if(os.path.exists(SubmitDir) == False):
    os.makedirs(SubmitDir)
sub.to_csv('%s/%s.csv' % (SubmitDir, OutputFileName), float_format='%.6f', index=False)
os.system('zip %s/%s.zip %s.csv' % (SubmitDir, OutputFileName, OutputFileName))

finish_time = datetime.datetime.now()
elapsed = (finish_time - start_time).seconds
print('\n======================')
print("CV score %.4f, Holdout score %.4f, Elapsed time: %.2fs" % (cv_score, holdout_score, elapsed))
print('======================\n')

----------------- CHECK -------------

============================ TrainData
           air_store_id visit_date  visitors    lgb_l1    lgb_l2  lgb_huber  \
0  air_ba937bf13d40fb24 2016-01-15  3.401197  3.642315  3.460873   3.468183   
1  air_ba937bf13d40fb24 2016-01-29  3.295837  3.642315  3.460873   3.468183   
2  air_ba937bf13d40fb24 2016-01-30  1.945910  2.946119  3.247834   3.241633   
3  air_ba937bf13d40fb24 2016-02-10  3.496508  2.972058  3.111297   3.101053   
4  air_ba937bf13d40fb24 2016-02-13  2.197225  3.136058  3.200620   3.217364   

   lgb_fair        en  lassolars  fold  
0  3.457178  3.475636   3.475966     0  
1  3.457178  3.475636   3.475966     0  
2  3.260333  3.257635   3.257384     0  
3  3.098959  3.115997   3.116237     0  
4  3.238128  3.237136   3.236274     0  

============================ HoldoutData
           air_store_id visit_date  visitors    lgb_l1    lgb_l2  lgb_huber  \
0  air_ba937bf13d40fb24 2016-01-13  3.258097  3.100422  3.103477   3.116652   
1

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


fold 0: valid score 0.497869, holdout score 0.499303, valid length 45371
saving for 0th fold data done.
fold 1: valid score 0.498573, holdout score 0.499272, valid length 45371
saving for 1th fold data done.
fold 2: valid score 0.495989, holdout score 0.499791, valid length 45371
saving for 2th fold data done.
fold 3: valid score 0.498530, holdout score 0.500248, valid length 45370
saving for 3th fold data done.
fold 4: valid score 0.496956, holdout score 0.500283, valid length 45370
saving for 4th fold data done.

CV score 0.4976, Holdout score 0.4998, Elapsed time: 14.00s

